<a href="https://colab.research.google.com/github/emerdg/Narrador-Interativo---Alura-Gemini/blob/master/Narrador_Interativo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Primeiro passo:
Insira suas `API KEY` do Gemini no campo `"Secrets"`, para o chatboot possa funcionar corretamente.

# Segundo passo:
Rode o código abaixo para o Colab possa preparar os bastidores.

(Não se preocupe, o Colab é um ferramenta Google, e ela não vai roubar a sua API KEY!)

---



In [ ]:
import os
import time
import sys

################################################################################
def digitar(texto):                   #Efeito de digitação, cortesia do Gemini!
    for letra in texto:
        sys.stdout.write(letra)
        sys.stdout.flush()
        time.sleep(0.01)              # velocidade de digitação
    print()                           # Adiciona uma nova linha ao final
################################################################################

digitar(texto='Pera ai, deixa eu pegar umas coisas aqui...')
!pip install -q google-genai          # Instalando Bibliotéca do Gemini

digitar(texto=
        '''\"Oi, google? Você deixa eu usar essas coisas aqui?
        \n Rapidinho, coisa boba... \"
        \n
        '''
        )
!pip install -q google-adk            # Instalando Bibliotéca do ADK (Agent Development Kit)


# Inserindo essas frases bobas a cada etapa eu consigo saber em que etapa o
# codigo se encontra ;)

from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY') # Copiando sua API KEY para a memória;


################################################################################
################################### Shor de Importação! Carrega tudo aqui, bora!

digitar(texto='Chegou! Deixa eu colocar cada coisa em seu lugar, só um segundo...')

from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")
################################################################################


digitar(texto=
    '''Tudo certo!
    \n Agora o Google nos emprestou o Gemini por um tempo. Vamos para o próximo passo!
    \n Você já pode executar o próximo bloco.
    '''
    )

In [ ]:
modelo_grande = "gemini-2.0-flash"
modelo_rapido = "gemini-2.0-flash-lite"

# Gerenciador de agentes
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

################################################################################ AGENTRE CONFIGURACAO DO NARRADOR
def agente_conf_narrador(descricao, referencias):
  conf_narrador = Agent(
      name="agente_conf_narrador",
      model=modelo_rapido,
      description="Agente responsável por criar o nosso narrador.",
      tools=[google_search],
      instruction='''
      Você é um agente responsável por configurar um narrador, definir qual é seu humor, gostos e estilo de narrativa baseado nas intruções do leitor.
      você deve se basear nas seguintes informações:
      Use DESCRICAO para caracterizar e descrever o narrador.
      Use REFERENCIAS e usando goole (google_search) para recuperar informações sobre os gostos e interesses do narrador.
      Foque no que é mais importante para a narração do jogo.
      CASO o leitor não dê um nome, crie um nome para ele ou ela.
      Se faltar informação, crie algo bem divertido para o leitor.

      IMPORTANTE: De forma alguma o narrador deve ser sexualizado, menor de idade (em aparencia ou idade), vulgar ou violento.
      IMPORTANTE: De forma alguma o narrador deve ser sexualizado, menor de idade (em aparencia ou idade), vulgar ou violento.
      '''
  )
  entrada_do_agente_conf_narrador = f'''
  DESCRICAO: {descricao} \n
  REFERENCIAS: {referencias}
  '''
  res_conf_narrador = call_agent(conf_narrador, entrada_do_agente_conf_narrador)
  return res_conf_narrador

  ################################################################################ AGENTRE CONFIGURACAO DA HISTORIA
def agente_conf_historia(onde, referencias, estilo, comportamento):
  conf_historia = Agent(
      name="agente_conf_historia",
      model=modelo_grande,
      description="Agente responsável por criar o cenário e a história.",
      tools=[google_search],
      instruction='''
      Você é um agente responsável por configurar a história que nosso narrador irá contar, definir aonde ela irá se passar, que gênero narrativo ela deve seguir e qual o foco da narrativa.
      você deve se basear nas seguintes informações:
      Use LUGAR para saber em que lugar se passa essa história.
      Use REFERENCIAS e usando goole (google_search) para recuperar informações que te ajudem a construir este cenario da melhor maneira possível
      Use ESTILO para adaptar a história a um estilo literário coerente e usando goole (google_search) encontre bons exemplos.
      Use COMPORTAMENTO para entender como essa hsitória será contada, seja como um narrador de um RPG de mesa, um livro jogo com alternativas ou de outra forma solicitada pelo leitor.
      Crie lugares e personágens importantes que podem ou não ser apresentados na história.
      Foque no que é mais importante para a narração da história, dando a melhor atmosfera possível para o ambiente e vida ao jogo.
      Se faltar informação, crie algo bem divertido para o leitor.

      IMPORTANTE: De forma alguma o ambiente ou os personagens devem ser sexualizados, vulgarizados ou apresentar violência além do necessário para uma boa narrativa.
      IMPORTANTE: De forma alguma o ambiente ou os personagens devem ser sexualizados, vulgarizados ou apresentar violência além do necessário para uma boa narrativa
      '''
  )
  entrada_do_agente_conf_historia = f'''
  LUGAR: {onde}
  REFERENCIAS: {referencias}
  ESTILO: {estilo}
  COMPORTAMENTO: {comportamento}
  '''
  res_conf_historia = call_agent(conf_historia, entrada_do_agente_conf_historia)
  return res_conf_historia

################################################################################ AGENTRE CRIACAO DO NARRADOR
def agente_narrador(res_conf_narrador, res_conf_historia, res_conf_personagem):
  agente_narrador = Agent(
      name="agente_narrador",
        model=modelo_rapido,
      description="Agente responsável por contar a história.",
      instruction='''
      Você é um Narrador de histórias.

      IMPORTANTE: De forma alguma o narrador deve ser sexualizado, menor de idade (em aparencia ou idade), vulgar ou violento com jogador.
      IMPORTANTE: De forma alguma o narrador deve ser sexualizado, menor de idade (em aparencia ou idade), vulgar ou violento com jogador.
      '''
      )
  entrada_do_agente_narrador = f'''
  Este é você: {res_conf_narrador}; \n
  ------------------------------------------------------- \n
  Esta é a história que iremos contar: {res_conf_historia}; \n
  ------------------------------------------------------- \n
  Este é o personagem do jogador: {res_conf_personagem} \n
  ------------------------------------------------------- \n
  '''

  res_agente_narrador = call_agent(agente_narrador, entrada_do_agente_narrador)
  return res_agente_narrador

################################################################################ AGENTRE CONFIGURACAO DO PERSONAGEM
def agente_conf_personagem(Persona, referencias, relacoes):
  conf_personagem = Agent(
      name="agente_conf_personagem",
      model=modelo_grande,
      description="Agente responsável por criar o personagem jogador.",
      tools=[google_search],
      instruction='''
      Você é um agente responsável por configurar o personágem do jogador, podendo ser mais de um, de acordo com a solicitação do leitor.
      Você deve se basear nas seguintes informações:
      Use PERSONA para descrever o personágem, gostos e aparencia.
      Use REFERENCIAS e usando goole (google_search) para recuperar informações que te ajudem a construir este personágem da melhor maneira possível.
      Use RELACOES para localizar o personagem na história e estabelecer ligações dele com lugares e personagens importantes.
      Inclua alguns pertences relevantes, gostos pessoais, inimisades e relações com outros personagens da narrativa.
      Foque no que é mais importante para a narração da história, tornando o personagem parte vital do jogo, sendo um ou mais personagens.
      Se faltar informação, crie algo bem divertido para o leitor.

      IMPORTANTE: De forma alguma o ambiente ou os personagens devem ser sexualizados, vulgarizados ou apresentar violência além do necessário para uma boa narrativa.
      IMPORTANTE: De forma alguma o ambiente ou os personagens devem ser sexualizados, vulgarizados ou apresentar violência além do necessário para uma boa narrativa
      '''
  )

  entrada_do_agente_conf_personagem = f'''
  PERSONA: {Persona}
  REFERENCIAS: {referencias}
  RELACOES: {relacoes}
  '''
  res_conf_personagem = call_agent(conf_personagem, entrada_do_agente_conf_personagem)
  return res_conf_personagem

digitar(texto=
    '''Agora sim!
    \n Semrpe que você rodar o bloco abaixo uma nova história interativa vai se iniciar!
    \n Vamos começar a contar histórias.
    '''
    )

In [ ]:
print("🚀 Iniciando o Sistema de narrativa com os 4 Agentes 🚀")

descricao = input("\n Por favor, descreva quem será o narrador que irá contar a sua história. \n Inclua informações como humor, gostos e estilos. \n Isso ajudará a dar mais naturalidade e personalidade a forma como a história será contada. \n")
referencias = input("\n O que ele gosta de ler, assistir ou jogar? \n Não necessariamente isso será incluso na história, mas ajuda a dar personalidade ao nerrador \n")

# CRIANDO NARRADOR ################################################
if not descricao:
  descricao = "Vamos criar um personagems aleatoreamente"

if not referencias:
  referencias = "Livros, jogos e animações da atualidade"

digitar("\n Ok, só um minuto, estou montando o narrador... \n Encaixando algumas peças, lendo o manual...")

final_narrador = agente_conf_narrador(descricao, referencias)

print("\n ---------------------------------------------------")

 ###############################################################################



onde = input("\n Onde se passará nossa história? \n Pode ser um lugar no mundo real ou na fantasia. \n")
referencias = input("\n Quer dar uma referencia de alguns mundos legais que você gosta? \n")
estilo =  input("\n Em que estilo de escrita devemos nos basear? \n literatura de horror, romance, young-adult, ficção científica? \n")
comportamento =  input("\n Como será nosso jogo? Um RPG, onde o narrador conta a história e você escreve ou escolhe a continuação? Terá conflitos como combates ou charadas? \n")

# CRIANDO HISTORIA ################################################
if not onde:
  onde = "Vamos criar um mundo interessante para você"

if not referencias:
  referencias = "Os mundos de fantasia mais famosos"

if not estilo:
  estilo = "RPG e romances da atualidade"

if not comportamento:
  comportamento = "Jogo de escolha de caminho, onde o narrador me dá de 2 a 5 opções"

digitar("\n Tá cento, minha equipe de roteiristas está escrevendo nossa história. \n Só mais um segundo!")

final_historia = agente_conf_historia(onde, referencias, estilo, comportamento)

print("\n ---------------------------------------------------")

 ###############################################################################



Persona = input("\n Quem é o seu personagem neste mundo? \n Dê uma breve descrição dele. \n")
referencias = input("\n Quem, no mundo da ficção, se parece com o seu personágem? \n")

# CRIANDO HISTORIA ################################################
if not Persona:
  Persona = "Vamos criar um pernagem interessante para você"

if not referencias:
  referencias = "personágens mais interessantes da atualidade"

digitar("\n Legal, agora nossa equipe de maquiagem e figurino vai trabalhar! \n Mais um segundo, por favor...")

final_personagem = agente_conf_personagem(Persona, referencias, final_historia)

print("\n ---------------------------------------------------")


################################################################################ INICIO DA NARRATIVA

print("\n OK! Vamos começar!")

client = genai.Client()

chat_config = types.GenerateContentConfig(
    system_instruction = f'''
    Você é um narrador de histórias, e vamos jogar um jogo de narrativa.
    Você vai descrever brevemente as cenas desta história e eu vou jogar com você de acordo com suas instruções.

    você é {final_narrador},
    ----------------------------
    A história é essa: {final_historia};
    ----------------------------
    O personágem do jogador é este:
    {final_personagem};
    ----------------------------

    IMPORTANTE: Seja criativo, descreva com cuidado as cenas da história, mas seja breve, tente se limitar a no máximo 6 parágrafos por cena.

    IMPORTANTE: De forma alguma o narrador deve ser sexualizado, menor de idade (em aparencia ou idade), vulgar ou violento com jogador.
    IMPORTANTE: De forma alguma o narrador deve ser sexualizado, menor de idade (em aparencia ou idade), vulgar ou violento com jogador.

    '''
    )

prompt = ('''
    Primeiro se apresente, descreva-se em um parágrafo;
    Depois descreva em dois paragrafos o mundo em que nossa história se passará;
    Enfim descreva em dois parágrafos o meu personágem, onde ele se encontra no mundo, e comece a contar a história;
    ''')
chat = client.chats.create(model=modelo_rapido, config=chat_config)

while prompt != "!FIM":
  resposta = chat.send_message(prompt)
  texto = ("\n NARRADOR : " + resposta.text + "\n\n")
  digitar(texto)
  prompt = input("SUA VEZ:")
